# STEP 3 of Segmenting and Clustering Neighborhoods in Toronto
---

## 3. Explore and cluster the neighborhoods in Toronto

## 3.a) initialization of the dataset from the previous step

Neighborhood has a total of 10 boroughs and 103 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need the dataset previously prepared.


Import all dependicies that we need

In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#1st Step of the assignement


website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# find the table class 'wikitable sortable'
soup = BeautifulSoup(website_url,'lxml')
Table_M_Toronto = soup.find('table',{'class':'wikitable sortable'})

#Datas under 'wikitable sortable'
datas_rows = Table_M_Toronto.findAll('tr')

#extract all datas to a list
list_toronto_M =[]
for data_row in datas_rows:
    cols=data_row.find_all('td')
    cols=[x.text.strip() for x in cols]
    list_toronto_M.append(cols)
#delete the empty first row from the title before to convert into dataframe    
list_toronto_M.pop(0)     

#Convert list into dataframe
df_toronto_M= pd.DataFrame(list_toronto_M,columns=['PostalCode','Borough','Neighborhood'])
#delete all rows where borough is not assigned
df_toronto_M = df_toronto_M[df_toronto_M.Borough != 'Not assigned']
df_toronto_M=df_toronto_M.reset_index(drop=True)
#Assigning value for neighbordhood is not assigned
df_toronto_M.loc[(df_toronto_M.Neighborhood == 'Not assigned'),'Neighborhood']=df_toronto_M.Borough

#2nd Step of the assignement
#read the csv file Geospatial_Coordinates.csv
df_coordinate = pd.read_csv('Geospatial_Coordinates.csv')

#rename the column Postal Code
df_coordinate=df_coordinate.rename(columns={"Postal Code": "PostalCode"})

#set PostalCode as an index
df_coordinate_dict= pd.DataFrame(df_coordinate.set_index('PostalCode').T.to_dict('index'))

#add a name to the index (opitonal but usefull for the merge)
df_coordinate_dict=df_coordinate_dict.rename_axis('PostalCode')

#set PostalCode as an index
df_toronto_M = df_toronto_M.set_index('PostalCode')

#Merging 
neighborhoods = pd.merge(df_toronto_M,df_coordinate_dict[['Latitude','Longitude']], on='PostalCode')

#reset the index 
neighborhoods = neighborhoods.reset_index()

## 3.b) Explore the dataset

In [3]:
#result
neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


I make sure that the dataset has all 10 boroughs and 103 neighborhoods.

In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [5]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Creating a map of Toronto with neighborhoods superimposed on top.

In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Each circle mark to reveal the name of the neighborhood and its respective borough

In [29]:
North_York_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


Get the geographical coordinates of North York.

In [30]:
address = 'North York, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [31]:
# create map of Manhattan using latitude and longitude values
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(North_York_data['Latitude'], North_York_data['Longitude'], North_York_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North_York)  
    
map_North_York

Using the Foursquare API to explore the neighborhoods and segment them.

## 3.c) Explore the neighborhoods and segment them using the Foursquare API

Note : I use different file to import the credential for Foursquare API named Foursquares_Credential

In [32]:
import Foursquares_Credential

CLIENT_ID = Foursquares_Credential.CLIENT_ID  # your Foursquare ID
CLIENT_SECRET = Foursquares_Credential.CLIENT_SECRET # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [34]:
North_York_data.loc[0, 'Neighborhood']

'Parkwoods'

Get the neighborhood's latitude and longitude values.

In [36]:
neighborhood_latitude = North_York_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = North_York_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = North_York_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


GET request URL. Name your URL url.

In [37]:
# type your answer here
VERSION='20201230'
#neighborhood_latitude=40.87655077879964
#neighborhood_longitude=-73.91065965862981
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request and examine the resutls

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6039771450c1fe01f345e7d3'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [41]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-41-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [42]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## Explore Neighborhoods in North York

#### create a function to repeat the same process to all the neighborhoods in North York

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Dataset of all venues on each neighborhood 

In [45]:
# type your answer here

North_York_venues = getNearbyVenues(names=North_York_data['Neighborhood'],
                                   latitudes=North_York_data['Latitude'],
                                   longitudes=North_York_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


#### check the size of the resulting dataframe

In [48]:
print(North_York_venues.shape)
North_York_venues.head()

(247, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### check how many venues were returned for each neighborhood

In [49]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills,26,26,26,26,26,26
Downsview,15,15,15,15,15,15
"Fairview, Henry Farm, Oriole",64,64,64,64,64,64
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2


#### Unique categories can be curated from all the returned venues

In [50]:
print('There are {} uniques categories.'.format(len(North_York_venues['Venue Category'].unique())))

There are 100 uniques categories.


## Analyze Each Neighborhood

In [52]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Bookstore,Boutique,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fabric Shop,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#### Size

In [54]:
North_York_onehot.shape

(247, 101)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [57]:
North_York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Bookstore,Boutique,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fabric Shop,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.105263,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.00,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.0,0.000000,0.00,0.000000,0.052632,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,0.052632,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.040000,0.000000,0.000000,0.000000,0.000000,0.080000,0.04,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.040000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.040000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.080000,0.040000,0.000000,0.040000,0.000000,0.040000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.040000,0.000000,0.0,0.000000,0.04,0.000000,0.040000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.04,0.000000,0.040000,0.0,0.

### New Size

In [58]:
North_York_grouped.shape

(19, 101)

### Print each neighborhood along with the top 5 most common venues

In [60]:
num_top_venues = 5

for hood in North_York_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = North_York_grouped[North_York_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.11
1          Coffee Shop  0.11
2          Gas Station  0.05
3        Shopping Mall  0.05
4  Fried Chicken Joint  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3    Greek Restaurant  0.04
4   Indian Restaurant  0.04


----Don Mills----
                 venue  freq
0                  Gym  0.12
1           Beer Store  0.08
2                 Café  0.08
3          Coffee Shop  0.08
4  Japanese Restaurant  0.08


----Downsview----
                venue  freq
0       Grocery Store  0.13
1                Park  0.13
2    Business Service  0.13
3             Airport  0.07
4  A

#### Put in dataframe

function to sort the venues in descending order.

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### New dataframe and display the top 10 venues for each neighborhood.

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = North_York_grouped['Neighborhood']

for ind in np.arange(North_York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Gas Station,Shopping Mall,Fried Chicken Joint,Ice Cream Shop,Diner,Middle Eastern Restaurant,Deli / Bodega,Mobile Phone Shop
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store,Liquor Store,Park,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Greek Restaurant,Indian Restaurant,Japanese Restaurant,Juice Bar,Fast Food Restaurant,Liquor Store,Pharmacy
3,Don Mills,Gym,Beer Store,Café,Coffee Shop,Japanese Restaurant,Restaurant,Bike Shop,Caribbean Restaurant,Clothing Store,Sandwich Place
4,Downsview,Grocery Store,Park,Business Service,Airport,Athletics & Sports,Bank,Liquor Store,Baseball Field,Food Truck,Discount Store


##  Cluster Neighborhoods

## Run _k_-means to cluster the neighborhood into 5 clusters.

In [67]:
kclusters = 5

North_York_grouped_clustering = North_York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 3, 0, 2, 4])

### new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [69]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_York_merged = North_York_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

North_York_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Jewelry Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Massage Studio,Lounge
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,French Restaurant,Hockey Arena,Portuguese Restaurant,Financial or Legal Service,Intersection,Coffee Shop,Accessories Store,Liquor Store,Mobile Phone Shop
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Shoe Store,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Accessories Store,Vietnamese Restaurant,Sushi Restaurant
3,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Gym,Beer Store,Café,Coffee Shop,Japanese Restaurant,Restaurant,Bike Shop,Caribbean Restaurant,Clothing Store,Sandwich Place
4,M6B,North York,Glencairn,43.709577,-79.445073,3.0,Pizza Place,Pub,Bakery,Japanese Restaurant,Park,Accessories Store,Lingerie Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


### visualize the resulting clusters

In [75]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['Latitude'], North_York_merged['Longitude'], North_York_merged['Neighborhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[0-1],
        fill=True,
        fill_color=rainbow[0-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

Cluster 1

In [77]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 0, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Pizza Place,French Restaurant,Hockey Arena,Portuguese Restaurant,Financial or Legal Service,Intersection,Coffee Shop,Accessories Store,Liquor Store,Mobile Phone Shop
2,North York,0.0,Clothing Store,Shoe Store,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Accessories Store,Vietnamese Restaurant,Sushi Restaurant
3,North York,0.0,Gym,Beer Store,Café,Coffee Shop,Japanese Restaurant,Restaurant,Bike Shop,Caribbean Restaurant,Clothing Store,Sandwich Place
5,North York,0.0,Gym,Beer Store,Café,Coffee Shop,Japanese Restaurant,Restaurant,Bike Shop,Caribbean Restaurant,Clothing Store,Sandwich Place
6,North York,0.0,Golf Course,Dog Run,Pool,Mediterranean Restaurant,Fast Food Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Massage Studio,Lounge
7,North York,0.0,Bank,Coffee Shop,Gas Station,Shopping Mall,Fried Chicken Joint,Ice Cream Shop,Diner,Middle Eastern Restaurant,Deli / Bodega,Mobile Phone Shop
8,North York,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Bakery,Japanese Restaurant,Bank,Women's Store,Juice Bar,Greek Restaurant
9,North York,0.0,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Caribbean Restaurant,Miscellaneous Shop,Bar,Massage Studio,Lingerie Store,Park,Movie Theater
11,North York,0.0,Grocery Store,Park,Business Service,Airport,Athletics & Sports,Bank,Liquor Store,Baseball Field,Food Truck,Discount Store
13,North York,0.0,Grocery Store,Park,Business Service,Airport,Athletics & Sports,Bank,Liquor Store,Baseball Field,Food Truck,Discount Store


Cluster 2

In [78]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 1, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,Park,Food & Drink Shop,Jewelry Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Massage Studio,Lounge
16,North York,1.0,Park,Jewelry Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Massage Studio,Lounge,Liquor Store
22,North York,1.0,Park,Convenience Store,Flower Shop,Accessories Store,Juice Bar,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Massage Studio


CLuster 3

In [79]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 2, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North York,2.0,Gym,Intersection,Accessories Store,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Massage Studio


Cluster 4

In [80]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 3, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,3.0,Pizza Place,Pub,Bakery,Japanese Restaurant,Park,Accessories Store,Lingerie Store,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
10,North York,3.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Accessories Store,Liquor Store,Park,Movie Theater,Mobile Phone Shop,Miscellaneous Shop
14,North York,3.0,Park,Construction & Landscaping,Trail,Bakery,Basketball Court,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


Cluster 5

In [81]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 4, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,4.0,Baseball Field,Fabric Shop,Accessories Store,Jewelry Store,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mediterranean Restaurant,Massage Studio
